In [1]:
import warnings
warnings.filterwarnings("ignore")
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import time
import re


import torch as T



import torch.nn as nn
import torch.nn.functional as F


from model import Model


from data_util import config, data
from data_util.batcher import Batcher
from data_util.data import Vocab


from train_util import *
from torch.distributions import Categorical
from rouge import Rouge
from numpy import random
import argparse
import torchsnooper
import logging

# -------- Test Packages -------
from beam_search import *
import shutil
from tensorboardX import SummaryWriter


# Logger

In [2]:
from datetime import datetime as dt

def getLogger(loggerName, loggerPath):
    # 設置logger
    logger = logging.getLogger(loggerName)  # 不加名稱設置root logger
    logger.setLevel(logging.DEBUG)
    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s: - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')
    logging.Filter(loggerName)

    # 使用FileHandler輸出到文件
    directory = os.path.dirname(loggerPath)
    if not os.path.exists(directory):
        os.makedirs(directory)
    fh = logging.FileHandler(loggerPath)

    fh.setLevel(logging.DEBUG)
    fh.setFormatter(formatter)

    # 使用StreamHandler輸出到屏幕
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    ch.setFormatter(formatter)
    # 添加兩個Handler
    logger.addHandler(ch)
    logger.addHandler(fh)
    # Handler只啟動一次
    # 設置logger
    logger.info(u'logger已啟動')
    return logger

def removeLogger(logger):
    logger.info(u'logger已關閉')
    handlers = logger.handlers[:]
    for handler in handlers:
        handler.close()
        logger.removeHandler(handler)

# View batch data

In [3]:
def test_batch():
    vocab = Vocab(config.vocab_path, config.vocab_size)
    batcher = Batcher(config.train_data_path, vocab, mode='train',
                           batch_size=config.batch_size, single_pass=False)
    batch = batcher.next_batch()
    # with torchsnooper.snoop():
    while batch is not None:
        example_list = batch.example_list
        for ex in example_list:
            r = str(ex.original_review)
            s = str(ex.original_summary)
            k = str(ex.key_words)
            sent = ex.original_summary_sents
#             print("original_review_sents:", r)
            print("original_summary_sents : ", s)
            print("key_words : ", k)
            print('------------------------------------------------------------\n')
        batch = batcher.next_batch()        
# test_batch()

# Get Bin Information

In [4]:
with open(config.bin_info,'r',encoding='utf-8') as f:
    lines = f.readlines()
    [print(line) for line in lines]
    train_num = int(lines[0].split(":")[1])
    test_num = int(lines[1].split(":")[1])
    val_num = int(lines[2].split(":")[1])
    # f.write("train : %s\n"%(len(flit_key_train_df)))
    # f.write("test : %s\n"%(len(flit_key_test_df)))
    # f.write("valid : %s\n"%(len(flit_key_valid_df)))


train : 19241

test : 6413

valid : 6413



# Summary Encoder

In [5]:
from torchsummaryX import summary
from model import Encoder,Model
device = T.device("cuda" if T.cuda.is_available() else "cpu") # PyTorch v0.4.0
encoder = Encoder().to(device)    

vocab = Vocab(config.vocab_path, config.vocab_size)
batcher = Batcher(config.train_data_path, vocab, mode='train',
                       batch_size=config.batch_size, single_pass=False)
batch = batcher.next_batch()
enc_batch, enc_lens, enc_padding_mask, enc_key_batch, enc_key_lens, enc_key_padding_mask, enc_batch_extend_vocab, extra_zeros, context = get_enc_data(batch)
enc_batch = Model(False,'word2Vec',vocab).embeds(enc_batch) #Get embeddings for encoder input

summary(encoder, enc_batch, enc_lens) # encoder summary

           Kernel Shape  Output Shape   Params  Mult-Adds
Layer                                                    
0_lstm                -  [3950, 1024]  3334144    3325952
1_reduce_h  [1024, 512]     [16, 512]   524800     524288
2_reduce_c  [1024, 512]     [16, 512]   524800     524288
---------------------------------------------------------
                       Totals
Total params          4383744
Trainable params      4383744
Non-trainable params        0
Mult-Adds             4374528


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_lstm,-,"[3950, 1024]",3334144,3325952
1_reduce_h,"[1024, 512]","[16, 512]",524800,524288
2_reduce_c,"[1024, 512]","[16, 512]",524800,524288


# Summary Decoder

In [6]:
from torchsummaryX import summary
from model import Decoder,Model
from train_util import *
device = T.device("cuda" if T.cuda.is_available() else "cpu") # PyTorch v0.4.0
# decoder = Decoder().to(device)    

model = Model(False,'word2Vec',vocab)
vocab = Vocab(config.vocab_path, config.vocab_size)
batcher = Batcher(config.train_data_path, vocab, mode='train',
                       batch_size=config.batch_size, single_pass=False)
batch = batcher.next_batch()
enc_batch, enc_lens, enc_padding_mask, enc_key_batch, enc_key_lens, enc_key_padding_mask, enc_batch_extend_vocab, extra_zeros, context = get_enc_data(batch)
enc_batch = model.embeds(enc_batch) #Get embeddings for encoder input
enc_out, enc_hidden = model.encoder(enc_batch, enc_lens)

# train_batch_MLE
dec_batch, max_dec_len, dec_lens, target_batch = get_dec_data(batch)                        #Get input and target batchs for training decoder
step_losses = []
s_t = (enc_hidden[0], enc_hidden[1])                                                        #Decoder hidden states
# x_t 為decoder每一個time step 的batch input
x_t = get_cuda(T.LongTensor(len(enc_out)).fill_(2))                             #Input to the decoder
prev_s = None                                                                               #Used for intra-decoder attention (section 2.2 in DEEP REINFORCED MODEL - https://arxiv.org/pdf/1705.04304.pdf)
sum_temporal_srcs = None     
             
    
for t in range(min(max_dec_len, config.max_dec_steps)):
    use_gound_truth = get_cuda((T.rand(len(enc_out)) > 0.25)).long()                        #Probabilities indicating whether to use ground truth labels instead of previous decoded tokens
    # use_gound_truth * dec_batch[:, t] : 為ground true time step token
    # (1 - use_gound_truth) * x_t : 為previous time step token
    x_t = use_gound_truth * dec_batch[:, t] + (1 - use_gound_truth) * x_t                   #Select decoder input based on use_ground_truth probabilities
    x_t = model.embeds(x_t)
#     final_dist, s_t, ct_e, sum_temporal_srcs, prev_s = model.decoder(x_t, s_t, enc_out, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, sum_temporal_srcs, prev_s)
    final_dist, s_t, ct_e, sum_temporal_srcs, prev_s = model.decoder(
    x_t, s_t, enc_out, enc_padding_mask,context, 
    extra_zeros,enc_batch_extend_vocab,sum_temporal_srcs, prev_s, 
    enc_key_batch, enc_key_lens)
    
    decoder_summary = summary(model.decoder, x_t, s_t, enc_out, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, sum_temporal_srcs, prev_s,enc_key_batch, enc_key_lens) # encoder summary
    break
decoder_summary

                               Kernel Shape     Output Shape    Params  \
Layer                                                                    
0_x_context                     [1324, 300]        [16, 300]    397500   
1_lstm                                    -        [16, 512]   1667072   
2_enc_attention.Linear_W_h     [1024, 1024]  [16, 845, 1024]   1048576   
3_enc_attention.Linear_W_s     [1024, 1024]       [16, 1024]   1049600   
4_enc_attention.Linear_v          [1024, 1]     [16, 845, 1]      1024   
5_dec_attention.Linear_W_prev    [512, 512]     [16, 1, 512]    262144   
6_dec_attention.Linear_W_s       [512, 512]        [16, 512]    262656   
7_dec_attention.Linear_v           [512, 1]       [16, 1, 1]       512   
8_p_gen_linear                    [2860, 1]          [16, 1]      2861   
9_V                             [2048, 512]        [16, 512]   1049088   
10_V1                          [512, 50000]      [16, 50000]  25650000   

                               Mult-A

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_x_context,"[1324, 300]","[16, 300]",397500,397200
1_lstm,-,"[16, 512]",1667072,1662976
2_enc_attention.Linear_W_h,"[1024, 1024]","[16, 845, 1024]",1048576,1048576
3_enc_attention.Linear_W_s,"[1024, 1024]","[16, 1024]",1049600,1048576
4_enc_attention.Linear_v,"[1024, 1]","[16, 845, 1]",1024,1024
5_dec_attention.Linear_W_prev,"[512, 512]","[16, 1, 512]",262144,262144
6_dec_attention.Linear_W_s,"[512, 512]","[16, 512]",262656,262144
7_dec_attention.Linear_v,"[512, 1]","[16, 1, 1]",512,512
8_p_gen_linear,"[2860, 1]","[16, 1]",2861,2860


# Train object

In [7]:
# from torchsummaryX import summary
class Train(object):
    def __init__(self, opt, vocab):
#         self.vocab = Vocab(config.vocab_path, config.vocab_size)
        self.vocab = vocab
        self.train_batcher = Batcher(config.train_data_path, self.vocab, mode='train',
                               batch_size=config.batch_size, single_pass=False)
        self.test_batcher = Batcher(config.test_data_path, self.vocab, mode='eval',
                               batch_size=config.batch_size, single_pass=True)
        self.opt = opt
        self.start_id = self.vocab.word2id(data.START_DECODING)
        self.end_id = self.vocab.word2id(data.STOP_DECODING)
        self.pad_id = self.vocab.word2id(data.PAD_TOKEN)
        self.unk_id = self.vocab.word2id(data.UNKNOWN_TOKEN)
        time.sleep(5)

    def save_model(self, iter, loss, r_loss):
        if not os.path.exists(config.save_model_path):
            os.makedirs(config.save_model_path)
        file_path = "/%07d_%.2f_%.2f.tar" % (iter, loss, r_loss)
        save_path = config.save_model_path + '/%s' % (self.opt.word_emb_type)
        if not os.path.isdir(save_path): os.mkdir(save_path)
        save_path = save_path + file_path
        T.save({
            "iter": iter + 1,
            "model_dict": self.model.state_dict(),
            "trainer_dict": self.trainer.state_dict()
        }, save_path)
        return file_path

    def setup_train(self):
        self.model = Model(opt.pre_train_emb, opt.word_emb_type, self.vocab)
        #         print("Model : ",self.model)
        #         logger.info("Model : ")
        logger.info(str(self.model))
        #         print("Encoder : ",self.model.encoder)
        #         print("Decoder : ",self.model.decoder)
        #         print("Embeds : ",self.model.embeds)
        self.model = get_cuda(self.model)
        device = T.device("cuda" if T.cuda.is_available() else "cpu")  # PyTorch v0.4.0
        if opt.multi_device:
            if T.cuda.device_count() > 1:
                #                 print("Let's use", T.cuda.device_count(), "GPUs!")
                logger.info("Let's use " + str(T.cuda.device_count()) + " GPUs!")
                self.model = nn.DataParallel(self.model, list(range(T.cuda.device_count()))).cuda()

        if isinstance(self.model, nn.DataParallel):
            self.model = self.model.module
        self.model.to(device)
        #         self.model.eval()

        self.trainer = T.optim.Adam(self.model.parameters(), lr=config.lr)
        start_iter = 0
        if self.opt.load_model is not None:
#             load_model_path = os.path.join(config.save_model_path, self.opt.load_model)
            load_model_path = config.save_model_path + self.opt.load_model
            print(load_model_path)
#             print('xxxx')
            checkpoint = T.load(load_model_path)
            start_iter = checkpoint["iter"]
            self.model.load_state_dict(checkpoint["model_dict"])
            self.trainer.load_state_dict(checkpoint["trainer_dict"])
            #             print("Loaded model at " + load_model_path)
            logger.info("Loaded model at " + load_model_path)
        if self.opt.new_lr is not None:
            self.trainer = T.optim.Adam(self.model.parameters(), lr=self.opt.new_lr)
        return start_iter

    def train_batch_MLE(self, enc_out, enc_hidden, enc_padding_mask, ct_e, extra_zeros, enc_batch_extend_vocab, enc_key_batch, enc_key_lens, batch):
        ''' Calculate Negative Log Likelihood Loss for the given batch. In order to reduce exposure bias,
                pass the previous generated token as input with a probability of 0.25 instead of ground truth label
        Args:
        :param enc_out: Outputs of the encoder for all time steps (batch_size, length_input_sequence, 2*hidden_size)
        :param enc_hidden: Tuple containing final hidden state & cell state of encoder. Shape of h & c: (batch_size, hidden_size)
        :param enc_padding_mask: Mask for encoder input; Tensor of size (batch_size, length_input_sequence) with values of 0 for pad tokens & 1 for others
        :param ct_e: encoder context vector for time_step=0 (eq 5 in https://arxiv.org/pdf/1705.04304.pdf)
        :param extra_zeros: Tensor used to extend vocab distribution for pointer mechanism
        :param enc_batch_extend_vocab: Input batch that stores OOV ids
        :param batch: batch object
        '''
        dec_batch, max_dec_len, dec_lens, target_batch = get_dec_data(
            batch)  # Get input and target batchs for training decoder
        step_losses = []
        s_t = (enc_hidden[0], enc_hidden[1])  # Decoder hidden states
        x_t = get_cuda(T.LongTensor(len(enc_out)).fill_(self.start_id))  # Input to the decoder
        prev_s = None  # Used for intra-decoder attention (section 2.2 in https://arxiv.org/pdf/1705.04304.pdf)
        sum_temporal_srcs = None  # Used for intra-temporal attention (section 2.1 in https://arxiv.org/pdf/1705.04304.pdf)
        try:
#             print('-----------------')
            for t in range(min(max_dec_len, config.max_dec_steps)):
                use_gound_truth = get_cuda((T.rand(len(enc_out)) > config.gound_truth_prob)).long()  # Probabilities indicating whether to use ground truth labels instead of previous decoded tokens
                x_t = use_gound_truth * dec_batch[:, t] + (1 - use_gound_truth) * x_t  # Select decoder input based on use_ground_truth probabilities
                x_t = self.model.embeds(x_t)                
                final_dist, s_t, ct_e, sum_temporal_srcs, prev_s = self.model.decoder(x_t, s_t, enc_out, enc_padding_mask,
                                                                                          ct_e, extra_zeros,
                                                                                          enc_batch_extend_vocab,
                                                                                          sum_temporal_srcs, prev_s, enc_key_batch, enc_key_lens)
                target = target_batch[:, t]
                log_probs = T.log(final_dist + config.eps)
                step_loss = F.nll_loss(log_probs, target, reduction="none", ignore_index=self.pad_id)
                step_losses.append(step_loss)
                x_t = T.multinomial(final_dist,1).squeeze()  # Sample words from final distribution which can be used as input in next time step
#                 print(config.vocab_size)
#                 print(x_t)
                is_oov = (x_t >= config.vocab_size).long()  # Mask indicating whether sampled word is OOV
                x_t = (1 - is_oov) * x_t.detach() + (is_oov) * self.unk_id  # Replace OOVs with [UNK] token
        except Exception as e:
            logger.error('xxxxxxxxxxx')
            traceback = sys.exc_info()[2]
            logger.error(sys.exc_info())
            logger.error(traceback.tb_lineno)
            logger.error(e)
#             logger.error(final_dist)
            logger.error('xxxxxxxxxxx')
#             print(step_loss)

                
        losses = T.sum(T.stack(step_losses, 1), 1)  # unnormalized losses for each example in the batch; (batch_size)
        batch_avg_loss = losses / dec_lens  # Normalized losses; (batch_size)
        mle_loss = T.mean(batch_avg_loss)  # Average batch loss
        return mle_loss

    def train_batch_RL(self, enc_out, enc_hidden, enc_padding_mask, ct_e, extra_zeros, enc_batch_extend_vocab,
                       review_oovs, greedy):
        '''Generate sentences from decoder entirely using sampled tokens as input. These sentences are used for ROUGE evaluation
        Args
        :param enc_out: Outputs of the encoder for all time steps (batch_size, length_input_sequence, 2*hidden_size)
        :param enc_hidden: Tuple containing final hidden state & cell state of encoder. Shape of h & c: (batch_size, hidden_size)
        :param enc_padding_mask: Mask for encoder input; Tensor of size (batch_size, length_input_sequence) with values of 0 for pad tokens & 1 for others
        :param ct_e: encoder context vector for time_step=0 (eq 5 in https://arxiv.org/pdf/1705.04304.pdf)
        :param extra_zeros: Tensor used to extend vocab distribution for pointer mechanism
        :param enc_batch_extend_vocab: Input batch that stores OOV ids
        :param review_oovs: Batch containing list of OOVs in each example
        :param greedy: If true, performs greedy based sampling, else performs multinomial sampling
        Returns:
        :decoded_strs: List of decoded sentences
        :log_probs: Log probabilities of sampled words
        '''
        s_t = enc_hidden  # Decoder hidden states
        x_t = get_cuda(T.LongTensor(len(enc_out)).fill_(self.start_id))  # Input to the decoder
        prev_s = None  # Used for intra-decoder attention (section 2.2 in https://arxiv.org/pdf/1705.04304.pdf)
        sum_temporal_srcs = None  # Used for intra-temporal attention (section 2.1 in https://arxiv.org/pdf/1705.04304.pdf)
        inds = []  # Stores sampled indices for each time step
        decoder_padding_mask = []  # Stores padding masks of generated samples
        log_probs = []  # Stores log probabilites of generated samples
        mask = get_cuda(T.LongTensor(len(enc_out)).fill_(
            1))  # Values that indicate whether [STOP] token has already been encountered; 1 => Not encountered, 0 otherwise

        for t in range(config.max_dec_steps):
            x_t = self.model.embeds(x_t)
            probs, s_t, ct_e, sum_temporal_srcs, prev_s = self.model.decoder(x_t, s_t, enc_out, enc_padding_mask, ct_e,
                                                                             extra_zeros, enc_batch_extend_vocab,
                                                                             sum_temporal_srcs, prev_s)
            if greedy is False:
                multi_dist = Categorical(probs)
                x_t = multi_dist.sample()  # perform multinomial sampling
                log_prob = multi_dist.log_prob(x_t)
                log_probs.append(log_prob)
            else:
                _, x_t = T.max(probs, dim=1)  # perform greedy sampling
            x_t = x_t.detach()
            inds.append(x_t)
            mask_t = get_cuda(T.zeros(len(enc_out)))  # Padding mask of batch for current time step
            mask_t[mask == 1] = 1  # If [STOP] is not encountered till previous time step, mask_t = 1 else mask_t = 0
            mask[(mask == 1) + (
            x_t == self.end_id) == 2] = 0  # If [STOP] is not encountered till previous time step and current word is [STOP], make mask = 0
            decoder_padding_mask.append(mask_t)
            is_oov = (x_t >= config.vocab_size).long()  # Mask indicating whether sampled word is OOV
            x_t = (1 - is_oov) * x_t + (is_oov) * self.unk_id  # Replace OOVs with [UNK] token

        inds = T.stack(inds, dim=1)
        decoder_padding_mask = T.stack(decoder_padding_mask, dim=1)
        if greedy is False:  # If multinomial based sampling, compute log probabilites of sampled words
            log_probs = T.stack(log_probs, dim=1)
            log_probs = log_probs * decoder_padding_mask  # Not considering sampled words with padding mask = 0
            lens = T.sum(decoder_padding_mask, dim=1)  # Length of sampled sentence
            log_probs = T.sum(log_probs,
                              dim=1) / lens  # (bs,)                                     #compute normalizied log probability of a sentence
        decoded_strs = []
        for i in range(len(enc_out)):
            id_list = inds[i].cpu().numpy()
            oovs = review_oovs[i]
            S = data.outputids2words(id_list, self.vocab, oovs)  # Generate sentence corresponding to sampled words
            try:
                end_idx = S.index(data.STOP_DECODING)
                S = S[:end_idx]
            except ValueError:
                S = S
            if len(S) < 2:  # If length of sentence is less than 2 words, replace it with "xxx"; Avoids setences like "." which throws error while calculating ROUGE
                S = ["xxx"]
            S = " ".join(S)
            decoded_strs.append(S)

        return decoded_strs, log_probs 

    def reward_function(self, decoded_sents, original_sents):
        rouge = Rouge()
        try:
            scores = rouge.get_scores(decoded_sents, original_sents)
        except Exception:
            #             print("Rouge failed for multi sentence evaluation.. Finding exact pair")
            logger.info("Rouge failed for multi sentence evaluation.. Finding exact pair")
            scores = []
            for i in range(len(decoded_sents)):
                try:
                    score = rouge.get_scores(decoded_sents[i], original_sents[i])
                except Exception:
                    #                     print("Error occured at:")
                    #                     print("decoded_sents:", decoded_sents[i])
                    #                     print("original_sents:", original_sents[i])
                    logger.info("Error occured at:")
                    logger.info("decoded_sents:", decoded_sents[i])
                    logger.info("original_sents:", original_sents[i])
                    score = [{"rouge-l": {"f": 0.0}}]
                scores.append(score[0])
        rouge_l_f1 = [score["rouge-l"]["f"] for score in scores]
        avg_rouge_l_f1 = sum(rouge_l_f1) / len(rouge_l_f1)
        rouge_l_f1 = get_cuda(T.FloatTensor(rouge_l_f1))
        return rouge_l_f1, scores, avg_rouge_l_f1

    # def write_to_file(self, decoded, max, original, sample_r, baseline_r, iter):
    #     with open("temp.txt", "w") as f:
    #         f.write("iter:"+str(iter)+"\n")
    #         for i in range(len(original)):
    #             f.write("dec: "+decoded[i]+"\n")
    #             f.write("max: "+max[i]+"\n")
    #             f.write("org: "+original[i]+"\n")
    #             f.write("Sample_R: %.4f, Baseline_R: %.4f\n\n"%(sample_r[i].item(), baseline_r[i].item()))


    def train_one_batch(self, batch,test_batch, iter):
        ans_list, batch_scores = None, None
        # Train
#         enc_batch, enc_lens, enc_padding_mask, enc_batch_extend_vocab, extra_zeros, context = get_enc_data(batch)

        enc_batch, enc_lens, enc_padding_mask, \
        enc_key_batch, enc_key_lens, enc_key_padding_mask,\
        enc_batch_extend_vocab, extra_zeros, context = get_enc_data(batch)

        enc_batch = self.model.embeds(enc_batch)  # Get embeddings for encoder input
        enc_key_batch = self.model.embeds(enc_key_batch)  # Get key embeddings for encoder input
        enc_out, enc_hidden = self.model.encoder(enc_batch, enc_lens)
        
        # Test
#         enc_batch2, enc_lens2, enc_padding_mask2, enc_batch_extend_vocab2, extra_zeros2, context2 = get_enc_data(test_batch)
        enc_batch2, enc_lens2, enc_padding_mask2, \
        enc_key_batch2, enc_key_lens2, enc_key_padding_mask2,\
        enc_batch_extend_vocab2, extra_zeros2, context2 = get_enc_data(test_batch)
    
        with T.autograd.no_grad():
            enc_batch2 = self.model.embeds(enc_batch2)
            enc_key_batch2 = self.model.embeds(enc_key_batch2)
            enc_out2, enc_hidden2 = self.model.encoder(enc_batch2, enc_lens2)
        # -------------------------------Summarization-----------------------
        if self.opt.train_mle == True:  # perform MLE training
            mle_loss = self.train_batch_MLE(enc_out, enc_hidden, enc_padding_mask, context, extra_zeros,
                                            enc_batch_extend_vocab, enc_key_batch, enc_key_lens, batch)
            mle_loss_2 = self.train_batch_MLE(enc_out2, enc_hidden2, enc_padding_mask2, context2, extra_zeros2,
                                            enc_batch_extend_vocab2, enc_key_batch2, enc_key_lens2, test_batch)
        else:
            mle_loss = get_cuda(T.FloatTensor([0]))
            mle_loss_2 = get_cuda(T.FloatTensor([0]))
        # original view
#         if opt.view:
#             sample_sents, ans_list = self.train_batch_decode(batch, enc_out, enc_hidden, enc_padding_mask, context,
#                                                              extra_zeros, enc_batch_extend_vocab, batch.rev_oovs,
#                                                              greedy=True)
#             rouge_l_f1, batch_scores, avg_rouge_l_f1 = self.reward_function(sample_sents, batch.original_summarys)
#             #             writer.add_text('Train/%s'% (iter), ans_list[0]['decoded_str'] , iter)
#             #             writer.add_text('Train/%s'% (iter), ans_list[0]['summary'] , iter)
#             #             writer.add_text('Train/%s'% (iter), ans_list[0]['review'] , iter)
#             writer.add_scalar('Train/avg_rouge_l_f1', avg_rouge_l_f1, iter)
            
        # --------------RL training-----------------------------------------------------
        if self.opt.train_rl == True:  # perform reinforcement learning training
            # multinomial sampling
            sample_sents, RL_log_probs = self.train_batch_RL(enc_out, enc_hidden, enc_padding_mask, context,
                                                             extra_zeros, enc_batch_extend_vocab, batch.rev_oovs,
                                                             greedy=False)
            sample_sents2, RL_log_probs2 = self.train_batch_RL(enc_out2, enc_hidden2, enc_padding_mask2, context2,
                                                             extra_zeros2, enc_batch_extend_vocab2, test_batch.rev_oovs,
                                                             greedy=False)
            with T.autograd.no_grad():
                # greedy sampling
                greedy_sents, _ = self.train_batch_RL(enc_out, enc_hidden, enc_padding_mask, context, extra_zeros,
                                                      enc_batch_extend_vocab, batch.rev_oovs, greedy=True)

            sample_reward, _, _ = self.reward_function(sample_sents, batch.original_summarys)
            baseline_reward, _, _ = self.reward_function(greedy_sents, batch.original_summarys)
            # if iter%200 == 0:
            #     self.write_to_file(sample_sents, greedy_sents, batch.original_abstracts, sample_reward, baseline_reward, iter)
            rl_loss = -(sample_reward - baseline_reward) * RL_log_probs  # Self-critic policy gradient training (eq 15 in https://arxiv.org/pdf/1705.04304.pdf)
            rl_loss = T.mean(rl_loss)

            batch_reward = T.mean(sample_reward).item()
            writer.add_scalar('Train_RL/RL_log_probs', RL_log_probs, iter)
        else:
            rl_loss = get_cuda(T.FloatTensor([0]))
            batch_reward = 0
        # ------------------------------------------------------------------------------------
        #         if opt.train_mle == True: 
        self.trainer.zero_grad()
        (self.opt.mle_weight * mle_loss + self.opt.rl_weight * rl_loss).backward()
        self.trainer.step()
        #-----------------------Summarization----------------------------------------------------
        if iter % 5000 == 0:
            with T.autograd.no_grad():
                train_rouge_l_f = self.calc_avg_rouge_result(iter,batch,'Train',enc_hidden, enc_out, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, enc_key_batch, enc_key_lens)
                test_rouge_l_f = self.calc_avg_rouge_result(iter,test_batch,'Test',enc_hidden2, enc_out2, enc_padding_mask2, context2, extra_zeros2, enc_batch_extend_vocab2, enc_key_batch2, enc_key_lens2)
                writer.add_scalars('Compare/rouge-l-f',  
                   {'train_rouge_l_f': train_rouge_l_f,
                    'test_rouge_l_f': test_rouge_l_f
                   }, iter)
                
#         return mle_loss.item(), batch_reward, ans_list, batch_scores
        return mle_loss.item(),mle_loss_2.item(), batch_reward

    def calc_avg_rouge_result(self, iter, batch, mode, enc_hidden, enc_out, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, enc_key_batch, enc_key_lens):
        pred_ids = beam_search(enc_hidden, enc_out, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, enc_key_batch, enc_key_lens, self.model, self.start_id, self.end_id, self.unk_id)

        decoded_sents = []
        ref_sents = []
        ref_sents2 = []
        article_sents = []
        
        summary_len = max_summary_len = long_seq_index = 0
        for i in range(len(pred_ids)):            
            decoded_words = data.outputids2words(pred_ids[i], self.vocab, batch.rev_oovs[i])
            if len(decoded_words) < 2:
                decoded_words = "xxx"
            else:
                decoded_words = " ".join(decoded_words)
            decoded_sents.append(decoded_words)
            summary = batch.original_summarys[i]
            summary2 = batch.original_summary_sents[i]
            review = batch.original_reviews[i]
            ref_sents.append(summary)
            ref_sents2.append(summary2)
            article_sents.append(review) 
            summary_len = len(summary.split(" "))
            if max_summary_len < summary_len: 
                max_summary_len = summary_len
                long_seq_index = i

        rouge = Rouge()    
        score = rouge.get_scores(decoded_sents, ref_sents, avg = True)    
        writer.add_scalars('%s/rouge-1' % mode,  # 'rouge-2' , 'rouge-l'
               {'f': score['rouge-1']['f'],
                'p': score['rouge-1']['p'],
                'r': score['rouge-1']['r']}
                , iter)
        writer.add_scalars('%s/rouge-2' % mode,  # 'rouge-2' , 'rouge-l'
               {'f': score['rouge-2']['f'],
                'p': score['rouge-2']['p'],
                'r': score['rouge-2']['r']}
                , iter)
        writer.add_scalars('%s/rouge-l' % mode,  # 'rouge-2' , 'rouge-l'
               {'f': score['rouge-l']['f'],
                'p': score['rouge-l']['p'],
                'r': score['rouge-l']['r']}
                , iter)
#         for i in range(len(decoded_sents)):
#             if type(article_sents[i]) != str: continue
#             if type(ref_sents[i]) != str:  continue
#             if type(decoded_sents[i]) != str:  continue

#         writer.add_text('Rouge/%s/%s' % (iter,mode), decoded_sents[long_seq_index], iter)
#         writer.add_text('Rouge/%s/%s' % (iter,mode), ref_sents[long_seq_index], iter)
#         #writer.add_text('Rouge/%s/%s' % (iter,mode), ref_sents2[long_seq_index], iter)
#         writer.add_text('Rouge/%s/%s' % (iter,mode), article_sents[long_seq_index], iter)
        
        for i in range(len(decoded_sents)):
#             writer.add_text('Rouge/%s/%s' % (iter,mode), decoded_sents[i], iter)
            writer.add_text('Rouge/%s/%s' % (iter,mode), ref_sents[i], iter)
            #writer.add_text('Rouge/%s/%s' % (iter,mode), ref_sents2[i], iter)
            writer.add_text('Rouge/%s/%s' % (iter,mode), article_sents[i], iter)
        
        return score['rouge-l']['f']
    
    def get_best_res_score(self, results, scores):
        max_score = float(0)
        _id = 0
        for idx in range(len(results)):
            re_matchData = re.compile(r'\-?\d{1,10}\.?\d{1,10}')
            data = re.findall(re_matchData, str(scores[idx]))
            score = sum([float(d) for d in data])
            if score > max_score:
                _id = idx
        return results[_id], scores[_id]

    def get_lr(self):
        for param_group in self.trainer.param_groups:
            return param_group['lr']

    def get_weight_decay(self):
        for param_group in self.trainer.param_groups:
            #             print(param_group)
            return param_group['weight_decay']

    def trainIters(self):
        final_file_path = None
        iter = self.setup_train()
        epoch = 0
        count = test_mle_total = train_mle_total = r_total = 0
        logger.info(u'------Training START--------')
        test_batch = self.test_batcher.next_batch()
        #         while iter <= config.max_iterations:
        while epoch <= config.max_epochs:
            train_batch = self.train_batcher.next_batch()
            try:
    #                 train_mle_loss, train_r, ans_list, batch_scores = self.train_one_batch(train_batch,test_batch, iter)
                train_mle_loss,test_mle_loss, r = self.train_one_batch(train_batch,test_batch, iter)

                #                 writer.add_scalar('lr', self.get_lr(), iter)
    #                 writer.add_scalar('Train/mle_loss', train_mle_loss, iter)
                writer.add_scalar('RL_Train/reward', r, iter)

    #                 writer.add_scalar('Test/mle_loss', test_mle_loss, iter)

                writer.add_scalars('Compare/mle_loss' ,  
                   {'train_mle_loss': train_mle_loss,
                    'test_mle_loss': test_mle_loss
                   }, iter)
                
#             # break
            except KeyboardInterrupt:
                logger.info("-------------------Keyboard Interrupt------------------")
                exit(0)
            except Exception as e:                
                logger.info("-------------------Ignore error------------------\n%s\n" % e)
                print("Please load final_file_path : %s" % final_file_path)
                traceback = sys.exc_info()[2]
                print(sys.exc_info())
                print(traceback.tb_lineno)
                print(e)
                break
            # if opt.train_mle == False: break
            train_mle_total += train_mle_loss
            r_total += r
            test_mle_total += test_mle_loss
            count += 1
            iter += 1

            if iter % 1000 == 0:
                train_mle_avg = train_mle_total / count
                r_avg = r_total / count
                test_mle_avg = test_mle_total / count
                epoch = int((iter * config.batch_size) / train_num) + 1
                logger.info('epoch: %s iter: %s train_mle_loss: %.3f test_mle_loss: %.3f reward: %.3f \n' % (epoch, iter, train_mle_avg, test_mle_avg, r_avg))

                count = test_mle_total = train_mle_total = r_total = 0
#                 writer.add_scalar('Train/mle_avg_loss', train_mle_avg, iter)
#                 writer.add_scalar('Test/mle_avg_loss', test_mle_avg, iter)
                writer.add_scalar('RL_Train/r_avg', r_avg, iter)
                
                writer.add_scalars('Compare/mle_avg_loss' ,  
                   {'train_mle_avg': train_mle_avg,
                    'test_mle_avg': test_mle_avg
                   }, iter)
            # break
            if iter % 5000 == 0:
                final_file_path = self.save_model(iter, test_mle_avg, r_avg)
#                 if opt.view:
#                     best_res, best_score = self.get_best_res_score(ans_list, batch_scores)
#                     logger.info('best_res: %s \n' % (best_res))
#                     logger.info('best_score: %s \n' % (best_score))
#                     writer.add_text('Train/%s' % (iter), best_res['decoded_str'], iter)
#                     writer.add_text('Train/%s' % (iter), best_res['summary'], iter)
#                     writer.add_text('Train/%s' % (iter), best_res['review'], iter)

In [8]:
def train_action(opt):
    try:       
        opt.rl_weight = 1 - opt.mle_weight  

        if opt.load_model:
            opt.load_model = "/%s/%s"%(opt.word_emb_type,opt.load_model)    

        logger.info(u'------Training Setting--------')  
   
        logger.info("Traing Type :%s" %(config.data_type))
        if opt.train_mle == True:
            logger.info("Training mle: %s, mle weight: %.2f"%(opt.train_mle, opt.mle_weight))

        if opt.train_rl == True:
            logger.info("Training rl: %s, rl weight: %.2f \n"%(opt.train_rl, opt.rl_weight))

        if opt.word_emb_type == 'bert': config.emb_dim = 768
        if opt.pre_train_emb : 
            logger.info('use pre_train_%s vocab_size %s \n'%(opt.word_emb_type,config.vocab_size))

        else:
            logger.info('use %s vocab_size %s \n'%(opt.word_emb_type,config.vocab_size))

        logger.info("intra_encoder: %s intra_decoder: %s \n"%(config.intra_encoder, config.intra_decoder))
        if opt.word_emb_type in ['word2Vec','glove']:
            config.vocab_path = config.Data_path + "Embedding/%s/word.vocab"%(opt.word_emb_type)            
#             config.vocab_size = len(open(config.vocab_path).readlines())
            vocab = Vocab(config.vocab_path, config.vocab_size)
        train_processor = Train(opt,vocab)
        train_processor.trainIters()
    except Exception as e:
        print(e)
        traceback = sys.exc_info()[2]
        logger.error(sys.exc_info())
        logger.error(traceback.tb_lineno)
        logger.error(e)
    logger.info(u'------Training END--------')  



In [9]:
from torch.autograd import Variable
import torch
def write_enc_graph():
    encoder_writer = SummaryWriter('runs/Pointer-Generator/glove/Encoder')
    device = T.device("cuda" if T.cuda.is_available() else "cpu") # PyTorch v0.4.0
    encoder = Encoder().to(device) 

    vocab = Vocab(config.vocab_path, config.vocab_size)
    batcher = Batcher(config.train_data_path, vocab, mode='train',
                           batch_size=config.batch_size, single_pass=False)
    batch = batcher.next_batch()
    enc_batch, enc_lens, enc_padding_mask, enc_key_batch, enc_key_lens, enc_key_padding_mask, enc_batch_extend_vocab, extra_zeros, context = get_enc_data(batch)
    enc_batch = Model(False,'word2Vec',vocab).embeds(enc_batch) #Get embeddings for encoder input

#     enc_batch = Variable(torch.rand(enc_batch.shape)).to(device) 
    enc_lens = torch.from_numpy(enc_lens).to(device) 

    encoder_writer.add_graph(encoder, (enc_batch, enc_lens), verbose=True)
    encoder_writer.close()

def write_dec_graph():
    decoder_writer = SummaryWriter('runs/Pointer-Generator/glove/Decoder')
    device = T.device("cuda" if T.cuda.is_available() else "cpu") # PyTorch v0.4.0
    # decoder = Decoder().to(device)    

    vocab = Vocab(config.vocab_path, config.vocab_size)
    model = Model(False,'word2Vec',vocab)
    
    batcher = Batcher(config.train_data_path, vocab, mode='train',
                           batch_size=config.batch_size, single_pass=False)
    batch = batcher.next_batch()
    enc_batch, enc_lens, enc_padding_mask, enc_key_batch, enc_key_lens, enc_key_padding_mask, enc_batch_extend_vocab, extra_zeros, context = get_enc_data(batch)
    enc_batch = model.embeds(enc_batch) #Get embeddings for encoder input
    enc_out, enc_hidden = model.encoder(enc_batch, enc_lens)

    # train_batch_MLE
    dec_batch, max_dec_len, dec_lens, target_batch = get_dec_data(batch)                        #Get input and target batchs for training decoder
    step_losses = []
    s_t = (enc_hidden[0], enc_hidden[1])                                                        #Decoder hidden states
    # x_t 為decoder每一個time step 的batch input
    x_t = get_cuda(T.LongTensor(len(enc_out)).fill_(2))                             #Input to the decoder
    prev_s = None                                                                               #Used for intra-decoder attention (section 2.2 in DEEP REINFORCED MODEL - https://arxiv.org/pdf/1705.04304.pdf)
    sum_temporal_srcs = None     


    for t in range(min(max_dec_len, config.max_dec_steps)):
        use_gound_truth = get_cuda((T.rand(len(enc_out)) > 0.25)).long()                        #Probabilities indicating whether to use ground truth labels instead of previous decoded tokens
        # use_gound_truth * dec_batch[:, t] : 為ground true time step token
        # (1 - use_gound_truth) * x_t : 為previous time step token
        if t == 0 :temp_batch = dec_batch[:, t]
        x_t = use_gound_truth * temp_batch + (1 - use_gound_truth) * x_t                   #Select decoder input based on use_ground_truth probabilities
        x_t = model.embeds(x_t)
    #     final_dist, s_t, ct_e, sum_temporal_srcs, prev_s = model.decoder(x_t, s_t, enc_out, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, sum_temporal_srcs, prev_s)
        final_dist, s_t, ct_e, sum_temporal_srcs, prev_s = model.decoder(
        x_t, s_t, enc_out, enc_padding_mask,context, 
        extra_zeros,enc_batch_extend_vocab,sum_temporal_srcs, prev_s, 
        enc_key_batch, enc_key_lens)   
        
#         (, , , , , , , , ,, ), verbose=True)
        '''
        intra_encoder = True   intra_decoder = True
        <class 'torch.Tensor'> <class 'tuple'> <class 'torch.Tensor'> <class 'torch.Tensor'>
        <class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>
        <class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'> <class 'numpy.ndarray'>
        '''
        print(type(x_t),type(s_t),type(enc_out),type(enc_padding_mask))
        print(type(context),type(extra_zeros),type(enc_batch_extend_vocab))
        print(type(sum_temporal_srcs),type(prev_s),type(enc_key_batch),type(enc_key_lens))

        #         decoder_summary = summary(model.decoder, x_t, s_t, enc_out, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, sum_temporal_srcs, prev_s,enc_key_batch, enc_key_lens) # encoder summary
#         x_t = Variable(torch.rand(x_t.shape)).to(device) 
        #             s_t = Variable(torch.rand(s_t.shape)).to(device)
#         enc_out = Variable(torch.rand(enc_out.shape)).to(device)
#         enc_padding_mask = Variable(torch.rand(enc_padding_mask.shape)).to(device,dtype=torch.long)
#         context = Variable(torch.rand(context.shape)).to(device)
#         extra_zeros = Variable(torch.rand(extra_zeros.shape)).to(device)
#         enc_batch_extend_vocab = Variable(torch.rand(enc_batch_extend_vocab.shape)).to(device)
        #             sum_temporal_srcs = Variable(torch.rand(sum_temporal_srcs.shape)).to(device)
        #             prev_s = Variable(torch.rand(prev_s.shape)).to(device)
#         enc_key_batch = Variable(torch.rand(enc_key_batch.shape)).to(device)
        enc_key_lens = torch.from_numpy(enc_key_lens).to(device) 
        
        decoder_writer.add_graph(model.decoder, 
                         (x_t, s_t, enc_out, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, sum_temporal_srcs, prev_s,enc_key_batch, enc_key_lens), verbose=True)
        
        decoder_writer.close()
        break    

In [10]:
# https://blog.csdn.net/u012869752/article/details/72513141
# 由于在jupyter notebook中，args不为空
from glob import glob
# nvidia-smi -pm 1
if __name__ == "__main__":   
    try:
        # --------------------------Training ----------------------------------
        parser = argparse.ArgumentParser()
        parser.add_argument('--train_mle', type=bool, default=True)
        parser.add_argument('--train_rl', type=bool, default=False)
        parser.add_argument('--mle_weight', type=float, default=1.0)
#         parser.add_argument('--load_model', type=str, default='/0044000_1.91_0.00.tar')
        parser.add_argument('--load_model', type=str, default=None)
        parser.add_argument('--new_lr', type=float, default=None)
        parser.add_argument('--multi_device', type=bool, default=True)
        parser.add_argument('--view', type=bool, default=True)
        parser.add_argument('--pre_train_emb', type=bool, default=True)
        parser.add_argument('--word_emb_type', type=str, default='glove')
        parser.add_argument('--train_action', type=bool, default=True)
        opt = parser.parse_args(args=[])
        
        today = dt.now()
        loggerPath = "LOG/%s-(%s_%s_%s)-(%s:%s:%s)"%(opt.word_emb_type,
                  today.year,today.month,today.day,
                  today.hour,today.minute,today.second)

        logger = getLogger(config.loggerName,loggerPath)   
        
        if opt.load_model == None:
            shutil.rmtree('runs/Pointer-Generator/glove', ignore_errors=True) # clear previous 
            shutil.rmtree('runs/Pointer-Generator/glove/exp-4', ignore_errors=True) # clear previous 
            shutil.rmtree('runs/Pointer-Generator/glove/Eecoder', ignore_errors=True) # clear previous 
            shutil.rmtree('runs/Pointer-Generator/glove/Decoder', ignore_errors=True) # clear previous 

                
        writer = SummaryWriter('runs/Pointer-Generator/glove/exp-4')
        
#         write_enc_graph()
#         write_dec_graph()
        if opt.train_action: train_action(opt)
        # --------------------------Testing ----------------------------------
#         parser = argparse.ArgumentParser()
#         parser.add_argument("--task", type=str, default="validate", choices=["validate","test"])
#         parser.add_argument("--start_from", type=int, default="0020000")
# #         parser.add_argument("--load_model", type=str, default=None)
#         parser.add_argument('--pre_train_emb', type=bool, default=True)
#         parser.add_argument('--word_emb_type', type=str, default='bert')
#         opt = parser.parse_args(args=[])                
#         if opt.word_emb_type == 'bert': config.emb_dim = 768
#         test_action(opt)

    except Exception as e:
        traceback = sys.exc_info()[2]
        print(sys.exc_info())
        print(traceback.tb_lineno)
        print(e)
    finally:
        removeLogger(logger)
        # export scalar data to JSON for external processing
        # tensorboard --logdir /home/eagleuser/Users/leyan/Text-Summarizer-FOP/TensorBoard
#         tensorboard --logdir ./runs
#         if not os.path.exists('TensorBoard'): os.makedirs('TensorBoard')
#         writer.export_scalars_to_json("TensorBoard/test.json")
        writer.close()
        

2020-01-23 16:32:12 - Text-Summary - INFO: - logger已啟動
2020-01-23 16:32:12 - Text-Summary - INFO: - ------Training Setting--------
2020-01-23 16:32:12 - Text-Summary - INFO: - Traing Type :Cameras
2020-01-23 16:32:12 - Text-Summary - INFO: - Training mle: True, mle weight: 1.00
2020-01-23 16:32:13 - Text-Summary - INFO: - use pre_train_glove vocab_size 50000 

2020-01-23 16:32:13 - Text-Summary - INFO: - intra_encoder: True intra_decoder: True 

2020-01-23 16:32:27 - Text-Summary - INFO: - Model(
  (encoder): Encoder(
    (lstm): LSTM(300, 512, batch_first=True, bidirectional=True)
    (reduce_h): Linear(in_features=1024, out_features=512, bias=True)
    (reduce_c): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (enc_attention): encoder_attention(
      (W_h): Linear(in_features=1024, out_features=1024, bias=False)
      (W_s): Linear(in_features=1024, out_features=1024, bias=True)
      (v): Linear(in_features=1024, out_features=1, bias=False)
    

Please load final_file_path : /0030000_3.42_0.00.tar
(<class 'RuntimeError'>, RuntimeError('CUDA error: device-side assert triggered',), <traceback object at 0x7fa1c010aa08>)
412
CUDA error: device-side assert triggered
